In [1]:
from pyspark.sql import SparkSession

scala_version = '2.12'  # TODO: Ensure this is correct
spark_version = '3.2.1'
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.2.0'
]
spark = SparkSession.builder\
   .master("local")\
   .appName("kafka-example")\
   .config("spark.jars.packages", ",".join(packages))\
   .getOrCreate()
spark

In [ ]:
# TODO: Replace with real list of data
data = ['Hello', 'World']
df = spark.createDataFrame([{'value': v} for v in data])

From the host, create the topic

```sh
kafka-topics --create --bootstrap-server localhost:9092 --topic foobar --partitions=1 --replication-factor=1
```

In [13]:
topic_name = 'foobar'
df.write.format("kafka")\
  .option("kafka.bootstrap.servers", "kafka:29092")\
  .option("topic", topic_name)\
  .save()

In [24]:
from pyspark.sql.functions import col, concat, lit

kafkaDf = spark.read.format("kafka")\
  .option("kafka.bootstrap.servers", "kafka:29092")\
  .option("subscribe", topic_name)\
  .option("startingOffsets", "earliest")\
  .load()
kafkaDf.select(
    concat(col("topic"), lit(':'), col("partition").cast("string")).alias("topic_partition"),
    col("offset"),
    col("value").cast("string")
).show()

+---------------+------+-----+
|topic_partition|offset|value|
+---------------+------+-----+
|       foobar:0|     0|Hello|
|       foobar:0|     1|World|
|       foobar:0|     2|Hello|
|       foobar:0|     3|World|
+---------------+------+-----+

